In [ ]:
import torch 
from transformers import BertTokenizer, BertForSequenceClassification
from torch.optim import AdamW
from torch.utils.data import DataLoader, Dataset
import torch.nn as nn

In [4]:
pretrained_model_name = 'bert-base-uncased'
tokenizer=BertTokenizer.from_pretrained(pretrained_model_name)
model=BertForSequenceClassification.from_pretrained(pretrained_model_name,num_labels=2)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [1]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

NameError: name 'torch' is not defined